#### UFLP Model for HW3 Q2 and Q3

In [2]:
using DataFrames
using XLSX
#xf = XLSX.readxlsx("6node.xlsx")
xf = XLSX.readxlsx("10node.xlsx")
XLSX.sheetnames(xf)
d = xf["Sheet1"]
#location = d["A2:E8"]
location = DataFrame(location[2:end,:],location[1,:])
distances = d["A15:G21"]
distances = DataFrame(distances[2:end,2:end], :auto);
display(distances)
display(location)

Row,x1,x2,x3,x4,x5,x6
,Any,Any,Any,Any,Any,Any
1,0.0,9.21954,3.0,7.07107,8.544,4.47214
2,9.21954,0.0,7.61577,5.0,4.47214,5.0
3,3.0,7.61577,0.0,7.28011,5.83095,4.12311
4,7.07107,5.0,7.28011,0.0,8.06226,3.16228
5,8.544,4.47214,5.83095,8.06226,0.0,6.08276
6,4.47214,5.0,4.12311,3.16228,6.08276,0.0


Row,Index,a,b,Demand,Fixed Cost
,Any,Any,Any,Any,Any
1,1,0.2,0.1,60,200
2,2,0.9,0.7,27,200
3,3,0.2,0.4,29,200
4,4,0.9,0.2,26,200
5,5,0.5,0.9,33,200
6,6,0.6,0.3,15,200


In [3]:

function calc_trans_cost(location,distances,status,assignment) #Given the opening of the inventory and the assignment, calculate the cost.
    transp = 0
    for i in 1:length(status)
        if status[i] == false 
            transp += distances[i,assignment[i]]*location[i,"Demand"]
        else
            transp += 0
        end
    end
    return transp
end

function UFLP_heuristic(location,distances)
    status = falses(nrow(location)) #this is indicate whether a index is a inventory or not. If true it is an inventory, else it is a demand
    assignment = zeros(nrow(location)) #this is to indicate where are the demands assigned to.
    cost = typemax(Int64)#Initialize the cost to the largest possible number.
    temp_cost = typemax(Int64)#A temporary cost for each iteration, it will contain the minimum cost for each iteration.
    terminal = true #if all next possible opening will incur a larger cost we terminate.
    num = 0 #this represents the number of inventory open
    temp_final_assignment = zeros(Int,nrow(location)) #final assigment at the end of each iteration
    temp_assignment = zeros(Int,nrow(location)) #we need a temp_assigment for each possible iteration.
    while terminal
        for i in 1 : length(status) 
            if !status[i] #try openning locations at the place where there is no inventory
                status[i] = true
                #Now we need to decide the assigment of the demands
                #println("we set ", i , "to be a inventory: ", status)
                for j in 1:length(status)
                    if !status[j]
                        temp_assignment[j] = parse(Int,string(findmin(distances[j,status])[2])[2:end]) #assign the demand j to the inventory with the lowest distance
                    elseif status[j]
                        temp_assignment[j] = 0
                    end
                end
                if temp_cost >= sum(status.*location[1,"Fixed Cost"])+ calc_trans_cost(location,distances,status,temp_assignment)#this is to calculate the best, need to add in transportation cost
                    #need to update tempcost and the assigment
                    temp_cost = sum(status.*location[1,"Fixed Cost"])+ calc_trans_cost(location,distances,status,temp_assignment)
                    temp_final_assignment = copy(temp_assignment)
                    #println("our assigment is ", temp_final_assignment, "our cost is ",temp_cost)
                end
                status[i] = false
            end
        end
        if cost > temp_cost
            cost = temp_cost
            assignment = copy(temp_final_assignment)
            status[findall(x->x == 0,assignment)] .= true
            num = sum(status)
            println("we open ", num, " inventories in total, the assigments are given here ", assignment,"the cost is ",cost, "the transportation cost is ", calc_trans_cost(location,distances,status,assignment))
        else
            terminal = false
        end
    end
    println("we open inventory at: ",findall(x->x==true,status),"our assigment for the demand is listed here: ",assignment,"our cost is: ", cost)
end
UFLP_heuristic(location,distances)

we open 1 inventories in total, the assigments are given here [6, 6, 6, 6, 6, 0]the cost is 1005.848603107106the transportation cost is 805.848603107106
we open 2 inventories in total, the assigments are given here [0, 6, 1, 6, 6, 0]the cost is 904.9503826642192the transportation cost is 504.95038266421915
we open 3 inventories in total, the assigments are given here [0, 5, 1, 6, 0, 0]the cost is 889.9668899493665the transportation cost is 289.96688994936653
we open inventory at: [1, 5, 6]our assigment for the demand is listed here: [0, 5, 1, 6, 0, 0]our cost is: 889.9668899493665


In [ ]:
#This is to simulate the heuristic swap algorithm


In [4]:
using JuMP
using Cbc

In [5]:
demand = location[!,"Demand"]
h = demand
f = [200,200,200,200,200,200]
c = Matrix(distances[1:6,1:6])


model = Model(Cbc.Optimizer)

# Define variables
@variable(model, x[1:6], Bin)
@variable(model, y[1:6, 1:6], Bin)

# Add constraint: sum of each columns has to be 1
@constraint(model, sum(y[:,j] for j in 1:6) .== 1)

# Add constraints: y[i,j] <= x[j] for all i, j
for i in 1:6
    for j in 1:6
        @constraint(model, y[i,j] <= x[j])
    end
end

# Define objective function
@objective(model, Min, sum(f[j] * x[j] for j in 1:6) +
                        sum(h[i] * sum(c[i,j] * y[i,j] for j in 1:6) for i in 1:6))

# Optimize the model
optimize!(model)

# Check optimization status and print solution if optimal
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found.")
    println("Solution:")
    println("x = ", value.(x))
    println("y = ", value.(y))
    for i in 1:6
        println(value.(y[i, :]))
    end
else
    println("No optimal solution found. Solver status: ", status)
end

Optimal solution found.
Solution:
x = [1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
y = [1.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 831.663 - 0.00 seconds
Cgl0004I processed model has 42 rows, 42 columns (42 integer (42 of which binary)) and 108 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 831.663
Cbc0038I Before mini branch and bound, 42 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiti

In [6]:
# Now we need to set capacity for each inventory
v = [60,60,60,60,60,60]
demand = location[!,"Demand"]
h = demand
f = [200,200,200,200,200,200]
c = Matrix(distances[1:6,1:6])

model = Model(Cbc.Optimizer)

# Define variables
@variable(model, x[1:6], Bin)
@variable(model, y[1:6, 1:6], Bin)

# Add constraint: sum of each columns has to be 1
@constraint(model, sum(y[:,j] for j in 1:6) .== 1)

# Add constraints: y[i,j] <= x[j] for all i, j
for i in 1:6
    for j in 1:6
        @constraint(model, y[i,j] <= x[j])
    end
end

# Add constraints: 
for j in 1:6
    @constraint(model, sum(h[:] .* y[:,j]) <= v[j])
end

# Define objective function
@objective(model, Min, sum(f[j] * x[j] for j in 1:6) +
                        sum(h[i] * sum(c[i,j] * y[i,j] for j in 1:6) for i in 1:6))

# Optimize the model
optimize!(model)

# Check optimization status and print solution if optimal
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found.")
    println("Solution:")
    println("x = ", value.(x))
    println("y = ", value.(y))
    for i in 1:6
        println(value.(y[i, :]))
    end
else
    println("No optimal solution found. Solver status: ", status)
end

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 871.176 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 30 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 18 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0004I processed model has 36 rows, 42 columns (42 integer (42 of which binary)) and 156 elements
Cbc0038I Initial state - 3 integers unsatisfied sum - 1.03448
Cbc0038I Pass   1: suminf.    0.90909 (5) obj. 956.627 iterations 11
Cbc0038I Solution found of 1110.03
Cbc0038I Rounding solution of 1002.97 is better than previous of 1110.03

Cbc0038I Before mini branch and bound, 37 integers at bound fixed and 0 continuous
Cbc0038I Full problem 36 rows 42 columns, reduced to 4 rows 3 columns
Cbc0038I Min

In [7]:
# Now we need to set capacity for each inventory
v = [60,60,60,60,60,60,60,60,60,60]
demand = location[!,"Demand"]
h = demand
f = [200,200,200,200,200,200,200,200,200,200]
c = Matrix(distances[1:6,1:6])

model = Model(Cbc.Optimizer)

# Define variables
@variable(model, x[1:6], Bin)
@variable(model, y[1:6, 1:6])

#Add constraint: all entries of y must be greater or equal to zero
for i in 1:6
    for j in 1:6
        @constraint(model, y[i,j] >= 0)
    end
end

# Add constraint: sum of each columns has to be 1
@constraint(model, sum(y[:,j] for j in 1:6) .== 1)

# Add constraints: y[i,j] <= x[j] for all i, j
for i in 1:6
    for j in 1:6
        @constraint(model, y[i,j] <= x[j])
    end
end

# Add constraints: 
for j in 1:6
    @constraint(model, sum(h[:] .* y[:,j]) <= v[j])
end

# Define objective function
@objective(model, Min, sum(f[j] * x[j] for j in 1:6) +
                        sum(h[i] * sum(c[i,j] * y[i,j] for j in 1:6) for i in 1:6))

# Optimize the model
optimize!(model)

# Check optimization status and print solution if optimal
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found.")
    println("Solution:")
    println("x = ", value.(x))
    println("y = ")
    for i in 1:6
        println(value.(y[i, :]))
    end
else
    println("No optimal solution found. Solver status: ", status)
end

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 871.176 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0004I processed model has 48 rows, 42 columns (6 integer (6 of which binary)) and 150 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.166667
Cbc0038I Pass   1: suminf.    0.16667 (1) obj. 945.639 iterations 6
Cbc0038I Solution found of 1112.31
Cbc0038I Relaxing continuous gives 1002.97
Cbc0038I Before mini branch and bound, 5 integers at bound fixed and 31 continuous
Cbc0038I Full problem 48 rows 42 columns, reduced to 5 rows 5 columns
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I Round again with cutoff of 997.234
Cbc0038I Pass   2: suminf.    0.16667 (1) obj. 945.639 iterations 0
Cbc0038I Pass   3: suminf.    0.07128 (1) obj. 997.234 iterations 11
Cbc0038I Pass   4: s

### Case Study: 